In [4]:
import pandas as pd
import numpy as np
# from data_prep.py import 

# Building stock data on house-level (BAG) 



This notebook uses the following datasets to merge ep-online(BAG) data with klimaatmonitordata:
- BAG data per house, including buildingtypes, retrieved from: [eponline](https://ep-online.nl/ep-online/) 
- Energy data per municipality, retrieved from: [klimaatmonitor](https://klimaatmonitor.databank.nl/Jive)
<br>
@author: Mark Hupkens
last edit: 13-05-2019

All code available on [github repo](https://github.com/markhupkens/EnergyTransitionModelling)

> __To do (16-05):__
1. fix column naming issue on import in vensim: add sheet name to column variables.
2. Check length of variables and their mapping --> buurten have more wijken as mapping than wijken have entities (see modelfix file)
3. Merge df_veh and df_solar to final dataframe

In [5]:
'''Import Data'''

# BAG data
df = pd.read_csv("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/masterdf.csv", error_bad_lines=False, sep =';') # handled data from https://www.rvo.nl/sites/default/files/2019/01/Voorlopige_labels_okt2018.zip

# Energy Data
df_energy_buurt = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor (energieverbruik , stadsverwarming, pv - Buurten,wijken en gemeenten 2017).xls",sheet_name='Buurt')
df_energy_gemeente = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor (energieverbruik , stadsverwarming, pv - Buurten,wijken en gemeenten 2017).xls",sheet_name='Gemeente')
df_energy_wijk = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor (energieverbruik , stadsverwarming, pv - Buurten,wijken en gemeenten 2017).xls",sheet_name='Wijk')

# Replace string values with nan value 0.424
df_energy_buurt.replace(to_replace='?',value=0.4242,inplace=True)
df_energy_buurt.replace(to_replace='-',value=0.4242,inplace=True)

df_energy_gemeente.replace(to_replace='?',value=0.4242,inplace=True)
df_energy_gemeente.replace(to_replace='-',value=0.4242,inplace=True)

df_energy_wijk.replace(to_replace='?',value=0.4242,inplace=True)
df_energy_wijk.replace(to_replace='-',value=0.4242,inplace=True)

### CHECK OUT NEW DATA
1. [Solar PV data per neighbourhood (CBS)](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=84517NED&_theme=287)
2. on chargingpoints (private)
3. [FEV and PHEV data on low scale](https://klimaatmonitor.databank.nl/Jive?workspace_guid=b341f460-f6d7-4c47-b74e-808f0e32e096)
4. [CBS data on vehicles per zipcode](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=37209hvv&_theme=439)

### 1. Solar PV data

In [5]:
'''1. Solar PV per neighborhood'''

# Solar PV data, url: https://www.cbs.nl/nl-nl/nieuws/2019/17/vermogen-zonnepanelen-meer-dan-de-helft-toegenomen
df_solar = pd.read_csv("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/Zonnestroom__wijken_en_buurten__2017_16052019_130403.csv", sep=';', error_bad_lines=False)

# clean data
df_solar['Regioaanduiding/Soort regio (omschrijving)'] = df_solar['Regioaanduiding/Soort regio (omschrijving)'].str.split(" ").str[0]

# Select levels
df_solar_buurt = df_solar.loc[df_solar['Regioaanduiding/Soort regio (omschrijving)']=='Buurt']
df_solar_wijk = df_solar.loc[df_solar['Regioaanduiding/Soort regio (omschrijving)']=='Wijk']
df_solar_gem = df_solar.loc[df_solar['Regioaanduiding/Soort regio (omschrijving)']=='Gemeente']

In [107]:
(len(df_solar.loc[np.isnan(df_solar['Aantal installaties bij woningen (aantal)'])==True])/len(df_solar))*100

20.05519227308177

### 2. Chargingpoints data (private)

### 3. FEV and PHEV

In [122]:
''' FEV and PHEV Data'''

df_fev = pd.read_excel('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor - Aantal geregistreerde  EV- PHEV2019 - Postcodes.xls',sheet_name='Auto s voertuigen 2019 Postcode')
df_voertuig = pd.read_csv('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/Motorvoertuigen_cbs_postcode.csv',error_bad_lines=False, sep=';')
df_pc = pd.read_csv('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/pc6hnr20170801_gwb.csv',sep=';')
df_pc_buurt = pd.read_csv('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/buurtnaam2017.csv',sep=';',encoding = 'unicode_escape')
df_pc_wijk = pd.read_csv('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/wijknaam2017.csv',sep=';',encoding = 'unicode_escape')
df_pc_gemeente = pd.read_csv('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/gemeentenaam2017.csv',sep=';',encoding = 'unicode_escape')

a. Zipcode coupling dataframe (CBS)

In [123]:
df_pc = df_pc.merge(df_pc_buurt, 
                    left_on='Buurt2017', 
                    right_on='BUURT2017').merge(df_pc_wijk, 
                                                left_on='Wijk2017', 
                                                right_on='WIJK2017').merge(df_pc_gemeente, 
                                                                           left_on='Gem2017', 
                                                                           right_on='GEM2017')

df_pc['PC4'] = df_pc['PC6'].str[:4]
df_pc.drop_duplicates(subset='PC4', keep='first', inplace=True)

b. Vehicles dataframe (CBS + Klimaatmonitor (RDW)

In [124]:
# Create merge columns
df_fev['Postcode'] = df_fev["Auto's/voertuigen 2019 - Postcodes"].str.split(" ").str[0]
df_voertuig['Postcode'] = df_voertuig['RegioS'].str.split(" ").str[0]
df_fev = df_fev.merge(df_voertuig,left_on='Postcode', right_on='Postcode', how='inner')

# Add buurt,wijk and gemeente names based on pc4
df_veh = df_fev.merge(df_pc, left_on='Postcode', right_on='PC4', how='inner')

In [176]:
'''Group data in wijken en gemeenten (pc4)'''

# Gemeenten
df_veh_gem = df_veh.groupby('GEMNAAM').agg({"Aantal geregistreerde elektrische personenauto's (FEV)":'sum',
                                                                              "Aantal geregistreerde plug-in hybride personenauto's (PHEV)":'sum',
                                                                              "Aantal geregistreerde personenauto's op aardgas (CNG)":'sum',
                                                                              "Aantal geregistreerde hybride personenauto's":'sum',
                                                                              "Personenauto's op waterstof (H2)":'sum',
                                                                             'Personenauto_2':'sum'})
df_veh_gem['Benzine_Diesel']=df_veh_gem['Personenauto_2']-(df_veh_gem["Aantal geregistreerde elektrische personenauto's (FEV)"] + 
                                                           df_veh_gem["Aantal geregistreerde plug-in hybride personenauto's (PHEV)"] + 
                                                           df_veh_gem["Aantal geregistreerde personenauto's op aardgas (CNG)"] + 
                                                           df_veh_gem["Aantal geregistreerde hybride personenauto's"] + 
                                                           df_veh_gem["Personenauto's op waterstof (H2)"])
# Wijken
df_veh_wijk = df_veh.groupby('WIJKNAAM').agg({"Aantal geregistreerde elektrische personenauto's (FEV)":'sum',
                                                                              "Aantal geregistreerde plug-in hybride personenauto's (PHEV)":'sum',
                                                                              "Aantal geregistreerde personenauto's op aardgas (CNG)":'sum',
                                                                              "Aantal geregistreerde hybride personenauto's":'sum',
                                                                              "Personenauto's op waterstof (H2)":'sum',
                                                                             'Personenauto_2':'sum'})
df_veh_wijk['Benzine_Diesel']=df_veh_wijk['Personenauto_2']-(df_veh_wijk["Aantal geregistreerde elektrische personenauto's (FEV)"] + 
                                                           df_veh_wijk["Aantal geregistreerde plug-in hybride personenauto's (PHEV)"] + 
                                                           df_veh_wijk["Aantal geregistreerde personenauto's op aardgas (CNG)"] + 
                                                           df_veh_wijk["Aantal geregistreerde hybride personenauto's"] + 
                                                           df_veh_wijk["Personenauto's op waterstof (H2)"])

### CONTINUE WITH OLD

In [6]:
print('number of neighbourhoods in bag', len(df['Neighbourhood Name'].unique()))
print('number of neighbourhoods in klimaatmonitor', 
      len(df_energy_buurt.Buurt.unique()))

number of neighbourhoods in bag 12040
number of neighbourhoods in klimaatmonitor 12252


## Data on buurt-level

Firstly, binary columns have to be created to allow for counting in a groupby dataframe. 13 new columns are added to show housing type (c1-c6) and label (A-G).

In [7]:
''' Create housing matrix'''

# Add housing matrix to enable building type count at the end of the script
df.loc[df['Housing Type']=='C1', 'Houses Detached BAG2018d'] = 1 
df.loc[df['Housing Type']=='C2', 'Houses 2u1Roof BAG2018d'] = 1 
df.loc[df['Housing Type']=='C3', 'Houses Corner BAG2018d'] = 1
df.loc[df['Housing Type']=='C4', 'Houses Row BAG2018'] = 1
df.loc[df['Housing Type']=='C5', 'Houses SingleFloorAppartments c5 BAG2018d'] = 1
df.loc[df['Housing Type']=='C6', 'Houses MultiFloorAppartments c6 BAG2018d'] = 1

# Add preliminary evaluation
df.loc[df['Preliminary Evaluation']=='A', 'Label A BAG2018d'] = 1 
df.loc[df['Preliminary Evaluation']=='B', 'Label B BAG2018d'] = 1 
df.loc[df['Preliminary Evaluation']=='C', 'Label C BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='D', 'Label D BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='E', 'Label E BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='F', 'Label F BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='G', 'Label G BAG2018d'] = 1

### 1. Group Data

In [8]:
'''Group adressdata in neighborhoods within municipalities'''

# group data
df_bag = df.groupby(['Municipality Name',
                     'Neighbourhood Name',
                     'Neighbourhood Code']).agg({'House No':'count',
                                              'Houses Detached BAG2018d':'count',
                                               'Houses 2u1Roof BAG2018d':'count',
                                              'Houses Corner BAG2018d':'count',
                                              'Houses Row BAG2018':'count',
                                              'Houses SingleFloorAppartments c5 BAG2018d':'count',
                                              'Houses MultiFloorAppartments c6 BAG2018d':'count',
                                              'Construction Year':'mean',
                                               'Label A BAG2018d':'count',
                                               'Label B BAG2018d':'count',
                                                'Label C BAG2018d':'count',
                                                'Label D BAG2018d':'count',
                                                'Label E BAG2018d':'count',
                                                'Label F BAG2018d':'count',
                                                'Label G BAG2018d':'count'})

# Rename column and duplicate index for merge later on
df_bag.rename(columns={'House No':'Houses All BAG2018d'})
df_bag['Neighbourhood Name_2'] = df_bag.index.get_level_values('Neighbourhood Name')
df_bag['Municipality Name_2'] = df_bag.index.get_level_values('Municipality Name')
df_bag['Neighbourhood Code_2'] = df_bag.index.get_level_values('Neighbourhood Code').map(str).str.split(".").str[0] # string values for easy merge

In [9]:
'''Group adressdata in districts within municipalities'''

# group data
df_bag_wijk = df.groupby(['Municipality Name',
                          'District Name',
                          'District Code']).agg({'House No':'count',
                                              'Houses Detached BAG2018d':'count',
                                               'Houses 2u1Roof BAG2018d':'count',
                                              'Houses Corner BAG2018d':'count',
                                              'Houses Row BAG2018':'count',
                                              'Houses SingleFloorAppartments c5 BAG2018d':'count',
                                              'Houses MultiFloorAppartments c6 BAG2018d':'count',
                                              'Construction Year':'mean',
                                               'Label A BAG2018d':'count',
                                               'Label B BAG2018d':'count',
                                                'Label C BAG2018d':'count',
                                                'Label D BAG2018d':'count',
                                                'Label E BAG2018d':'count',
                                                'Label F BAG2018d':'count',
                                                'Label G BAG2018d':'count'})

# Rename column and duplicate index for merge later on
df_bag_wijk.rename(columns={'House No':'Houses All BAG2018d'})
df_bag_wijk['District Name_2'] = df_bag_wijk.index.get_level_values('District Name')
df_bag_wijk['Municipality Name_2'] = df_bag_wijk.index.get_level_values('Municipality Name')
df_bag_wijk['District Code_2'] = df_bag_wijk.index.get_level_values('District Code').map(str).str.split(".").str[0] # string values for easy merge

### 2. Merge BAG with Klimaatmonitor

In [10]:
'''Buurt: merge grouped bag data with klimaatmonitordata on buurt'''

df_merged_buurt = df_bag.merge(df_energy_buurt,
                               left_on='Neighbourhood Name_2', 
                               right_on='Buurt',how='inner') # keep intersection of keys

df_merged_buurt = df_merged_buurt.groupby(['Municipality Name_2',
                                           'Neighbourhood Name_2',
                                           'Neighbourhood Code_2']).mean() # group by original index

In [11]:
'''Gemeente merge grouped bag data with klimaatmonitordata on Gemeente'''

df_merged_gemeente = df_bag.merge(df_energy_gemeente,
                                  left_on='Municipality Name', 
                                  right_on='Gemeente',
                                  how='inner') # keep intersection of keys

df_merged_gemeente = df_merged_gemeente.groupby(['Municipality Name_2']).mean() # group by original index

In [12]:
'''Wijk merge grouped bag data with klimaatmonitordata on wijk'''

df_merged_wijk = df_bag_wijk.merge(df_energy_wijk,
                                   left_on='District Name', 
                                   right_on='Wijk',
                                   how='inner') # keep intersection of keys

df_merged_wijk = df_merged_wijk.groupby(['Municipality Name_2',
                                         'District Name_2',
                                         'District Code_2']).mean() # group by original index

### 3. Merge data with Model-setup framework
Import modelsetup file and merge build-environment data with it on municipality name

In [13]:
# Remove special characters from column names
df_merged_gemeente.columns = df_merged_gemeente.columns.str.replace("[","").str.replace("]","")
df_merged_buurt.columns = df_merged_gemeente.columns.str.replace("[","").str.replace("]","")
df_merged_wijk.columns = df_merged_gemeente.columns.str.replace("[","").str.replace("]","")

In [14]:
# Import model setup files

df_mod_gemeente = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='gemeente')
df_mod_buurt = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='buurt')
df_mod_wijk = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='wijk') # wijkdata horrible from klimaatmonitor
df_mod_mod = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='ModelSpecification')

In [15]:
# split entity string to match building data on municipality name
df_mod_gemeente['Municipality Name'] = df_mod_gemeente["Entities"].str.split(" G").str[0] 
df_mod_wijk['Wijk Code'] = df_mod_wijk["Entities"].str.split(" W").str[-1].str.strip("K").str.strip(" ").str.lstrip("0") # Wijk on wijkcode, ditching leading 0's
df_mod_buurt['Buurt Code'] = df_mod_buurt["Entities"].str.split(" B").str[-1].str.strip("U").str.strip(" ").str.lstrip("0") # Buurt on buurt code, ditching leading 0's

In [16]:
# Check
len_mod_gem = len(df_mod_gemeente)
len_mod_buurt = len(df_mod_buurt)
len_mod_wijk = len(df_mod_wijk)

In [17]:
'''Merge prepared data with modelsetup data'''

# Gemeenten
df_mod_gemeente = df_mod_gemeente.merge(df_merged_gemeente, left_on='Municipality Name', right_on= df_merged_gemeente.index)
df_mod_gemeente = df_mod_gemeente.drop(['Municipality Name'], axis=1)

# Buurten
df_mod_buurt = df_mod_buurt.merge(df_merged_buurt, left_on='Buurt Code', right_on= df_merged_buurt.index.get_level_values('Neighbourhood Code_2'))
# df_mod_buurt = df_mod_buurt.drop(['Buurt Name'], axis=1)

# Wijken
df_mod_wijk = df_mod_wijk.merge(df_merged_wijk, left_on='Wijk Code', right_on=df_merged_wijk.index.get_level_values('District Code_2'))
# df_mod_wijk = df_mod_wijk.drop(['District Name_2'], axis=1)

In [15]:
print(len(df_mod_gemeente) - len(mod_gem),"Municipalities gained")
print(len(df_mod_buurt)- len(mod_buurt),"Buurten gained")
print(len(df_mod_wijk)- len(mod_wijk),"wijken gained")

-40 Municipalities gained
-732 Buurten gained
-223 wijken gained


In [16]:
print(len(df_mod_gemeente)) 
print(len(df_mod_buurt))
print(len(df_mod_wijk))

340
12573
2863


### 4. Export new modelspecification file

In [17]:
# export as xlsx to genereate new modelspecification file
from pandas import ExcelWriter

with pd.ExcelWriter('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL01_MH.xlsx') as writer:  # doctest: +SKIP
    df_mod_buurt.to_excel(writer, sheet_name='buurt')
    df_mod_wijk.to_excel(writer, sheet_name='wijk')
    df_mod_gemeente.to_excel(writer, sheet_name='gemeente')
    df_mod_mod.to_excel(writer, sheet_name='ModelSpecification')